# 1. Lógica de programación

**Instrucciones:** Dada la siguiente lista (celda de código de abajo), elimina los elementos duplicados y realiza una ordenación de la misma. Deberás de formular dos maneras diferentes de resolver el problema anterior para lo cual deberás de considerar las siguientes restricciones:

  a. No puedes utilizar librerías.
  
  b. Si ocupas alguna función de Python base en una forma de resolver el problema, no podrás ocuparla en la segunda formulación.

In [1]:
# Aqui coloca tu nombre
Nombre = 'Angel Josué Mejía Nájera'

In [4]:
lista = [1, -4, 0, 0, 2, 9]

In [3]:
# Solucion 1

#Función de ordenamiento
def Quick_Sort(l, low, high):
    if low >= high: #Condición para que no se cicle la recursión
        return
    pivot = MedianOfThree(l, low, high) #Generamos pivote con media de 3 valores
    lower = low + 1
    higher = high - 1
    while lower < higher: #Ciclos while que particionan ordenadamente el espacio dado el pivote
        while lower < higher and l[higher] > pivot:
            higher = higher - 1

        if lower < higher:
            l[lower] = l[higher]
            lower = lower + 1

        while lower < higher and l[lower] <= pivot:
            lower = lower + 1

        if lower < higher:
            l[higher] = l[lower]
            higher = higher - 1

        l[lower] = pivot

    Quick_Sort(l, low, lower - 1) #Recursión dado el espacio particionado por el pivote
    Quick_Sort(l, lower + 1, high)

#Función que determina la media dados 3 valores de una lista
#y ordena los valores en la correspondiente lista
def MedianOfThree(L, low, high):
    mid = (low + high)//2 #Calculamo valor medio de la lista
    #Ordenamos los 3 valores
    if L[high] < L[low]:  L[high], L[low] = L[low], L[high]
    if L[mid] < L[low]: L[mid], L[low] = L[low], L[mid]
    if L[high] < L[mid]: L[high], L[mid] = L[mid], L[high]
    if mid != low: L[low+1], L[mid] = L[mid], L[low+1] #Reposicionamos el valor del pivote despues del primer valor de la lista
    return L[low+1]

#Función principal de ordenamiento
def Sort1(l):
    if len(l) > 0: #Condicion necesaria de longitud
       Deduplicate(lista)
       Quick_Sort(lista, 0, len(l) - 1)

#Función que remueve duplicados de una lista
def Deduplicate(l):
    l[:] = list(dict.fromkeys(l)) #Se hace conversión a diccionario y se obtienen las keys únicas

#Main
Sort1(lista)
print(lista)

[-4, 0, 1, 2, 9]


In [8]:
# Solucion 2
lista = [1, -4, 0, 0, 2, 9]

#Función de ordenamiento
def Sort2(l):
    l[:] = sorted(set(l)) #Creamos un set para eliminar duplicados y luego usamos la función nativa sorted para ordenar

#Main
Sort2(lista)
print(lista)

[-4, 0, 1, 2, 9]


# 2. Uso de Pandas y SQL

A continuación generaremos información simulada para que puedas responder una serie de preguntas, para ello, solo ejecuta la celda de abajo:

In [9]:
# @title Generación de datos
# Importacion de las librerias e instalacion de libreria para generar datos falso
import pandas as pd
import random as r
%pip install Faker
from faker import Faker
fake = Faker()
import sqlite3 as sql
import warnings
warnings.filterwarnings('ignore')
import time

inicio = time.time()
# Catalogo de divisas y negocios
divisas = ['MXN', 'USD', 'EUR']
negocios = ['EMPRESAS', 'CORPORATIVA', 'GOBIERNO']

# Generamos primer dataframe con las columnas de numero de cliente, moneda y capital en moneda origen
n_cliente = [str(r.randint(0,9)) + str(r.randint(0,9)) + str(r.randint(0,9)) for i in range(1000)]
moneda = [r.choice(divisas)  for i in range(len(n_cliente))]
capital = [round(r.random() * r.randint(1, 100000), 2) for i in range(len(n_cliente))]

info_clientes = pd.DataFrame({
    'NUMERO_CLIENTE': n_cliente,
    'MONEDA': moneda,
    'CAPITAL MO': capital
})

# Eliminamos duplicados para unicidad en join posterior
info_clientes = info_clientes.drop_duplicates(subset='NUMERO_CLIENTE')

# Agregamos nombres de clientes
nom_cliente = [fake.company() for i in range(10000)]
nom_cliente = list(set(nom_cliente))[:225]
nom_cliente = [r.choice(nom_cliente) for i in range(info_clientes.shape[0])]
info_clientes['NOMBRE_CLIENTE'] = nom_cliente

# Creamos dataframe de negocios
negocio = pd.DataFrame(info_clientes['NUMERO_CLIENTE'])
negocio['NEGOCIO'] = [r.choice(negocios) for i in range(info_clientes.shape[0])]

# Exportamos para generar info y evaluar resultados
info_clientes.to_excel('info_clientes_' + Nombre + '.xlsx', index=False)
negocio.to_excel('Negocio_' + Nombre + '.xlsx', index=False)

################################################################################}
# Creacion de bbdd, tablas e insercion de la informacion
con = sql.connect('BBDD_MUESTRA.db')
cur = con.cursor()

query = """
  CREATE TABLE CLIENTES(
    NUMERO_CLIENTE INT,
    MONEDA TEXT,
    CAPITAL_MO REAL,
    NOMBRE_CLIENTE TEXT
  )
"""

cur.execute(query)
con.commit()
con.close()

con = sql.connect('BBDD_MUESTRA.db')
cur = con.cursor()
query = """
  CREATE TABLE NEGOCIO(
    NUMERO_CLIENTE INT,
    NEGOCIO TEXT
  )
"""

cur.execute(query)
con.commit()
con.close()

info_clientes.reset_index(drop=True, inplace=True)
con = sql.connect('BBDD_MUESTRA.db')
cur = con.cursor()
for i in range(len(info_clientes)):
  query = f"""
    INSERT INTO CLIENTES VALUES(
      {info_clientes.loc[i, 'NUMERO_CLIENTE']},
      '{info_clientes.loc[i, 'MONEDA']}',
      {info_clientes.loc[i, 'CAPITAL MO']},
      '{info_clientes.loc[i, 'NOMBRE_CLIENTE']}'
    )
  """

  cur.execute(query)

con.commit()
con.close()

negocio.reset_index(drop=True, inplace=True)
con = sql.connect('BBDD_MUESTRA.db')
cur = con.cursor()
for i in range(len(negocio)):
  query = f"""
    INSERT INTO NEGOCIO VALUES(
      {negocio.loc[i, 'NUMERO_CLIENTE']},
      '{negocio.loc[i, 'NEGOCIO']}'
    )
  """

  cur.execute(query)

con.commit()
con.close()

final = time.time()
print(f'Ejecución a tiempo: {round((final-inicio)/60, 2)} minutos')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
Ejecución a tiempo: 0.06 minutos


---

# Ejercicio 1

Con el modelo de datos generado con la celda de código anterior, tendremos un dataframe:

In [10]:
info_clientes.head()

,NUMERO_CLIENTE,MONEDA,CAPITAL MO,NOMBRE_CLIENTE
0,230,EUR,16595.14,"Clarke, Walter and Moody"
1,907,USD,34577.06,Wyatt-Wyatt
2,419,EUR,21747.49,Brock-Snyder
3,062,EUR,12962.23,Patrick PLC
4,240,MXN,996.61,Webb-Day


Donde la columna CAPITAL MO almacena el capital del cliente en moneda origen. De tal manera, usando los siguientes valores de tipo de cambio

* USD: 16.9623
* EUR: 18.305021

a. Crea una columna denominada CAPITAL VALORIZADO donde realices la conversión correspondiente del capital.

In [30]:
# Solucion
#Guardamos tipos de cambio
Tipos_Cambio = {'USD': 16.9623, 'EUR': 18.305021}

#Definimos función que convierte el capital según el tipo de cambio
def Rate_Convert(row):
  Tipo_Cambio = Tipos_Cambio.get(row['MONEDA'])
  if Tipo_Cambio is not None:
    return row['CAPITAL MO'] * Tipo_Cambio
  else:
    return row['CAPITAL MO']

#Aplicamos función sobre cada fila
info_clientes['CAPITAL VALORIZADO'] = info_clientes.apply(Rate_Convert, axis=1)
info_clientes.head()

,NUMERO_CLIENTE,MONEDA,CAPITAL MO,NOMBRE_CLIENTE,CAPITAL VALORIZADO
0,230,EUR,16595.14,"Clarke, Walter and Moody",303774.386198
1,907,USD,34577.06,Wyatt-Wyatt,586506.464838
2,419,EUR,21747.49,Brock-Snyder,398088.261147
3,062,EUR,12962.23,Patrick PLC,237273.892357
4,240,MXN,996.61,Webb-Day,996.610000


In [103]:
info_clientes['CAPITAL VALORIZADO'].sum()

200109913.04781052

b. ¿En qué moneda es donde se tiene la mayor concentración de capital y por qué?

In [75]:
# Solucion
#Creamos df donde agrupamos valores según el tipo de moneda de origen
Report = pd.DataFrame({
    'TOTAL CLIENTES': info_clientes.groupby(['MONEDA'])['NUMERO_CLIENTE'].count(),
    'PROMEDIO CAPITAL MO': info_clientes.groupby(['MONEDA'])['CAPITAL MO'].mean(),
    'PROMEDIO CAPITAL VALORIZADO': info_clientes.groupby(['MONEDA'])['CAPITAL VALORIZADO'].mean(),
    'CAPITAL VALORIZADO': info_clientes.groupby(['MONEDA'])['CAPITAL VALORIZADO'].sum()
}).sort_values(by=['CAPITAL VALORIZADO'], ascending=False)
Report.iloc[:, 1:4] = Report.iloc[:, 1:4].applymap("${:,.2f}".format)
print(Report)

#R: La mayor concentración de capital se encuentra en Euros. Esto se debe a la fortaleza de esta divisa en comparación con el peso mexicano. Además, la cartera de clientes está compuesta
#principalmente por aquellos cuya moneda de origen es el euro. Además, es notable que, aunque el promedio del capital retenido en la moneda de origen es bastante similar para las tres divisas,
#una vez que se realiza la conversión a pesos mexicanos (Promedio Capital Valorizado), las diferencias se vuelven significativas. Esto demuestra claramente la importancia de valorar el capital
#en la moneda del país receptor.


        TOTAL CLIENTES PROMEDIO CAPITAL MO PROMEDIO CAPITAL VALORIZADO  \
MONEDA                                                                   
EUR                218          $26,339.06                 $482,136.97   
USD                217          $24,324.36                 $412,597.10   
MXN                206          $26,555.75                  $26,555.75   

       CAPITAL VALORIZADO  
MONEDA                     
EUR       $105,105,858.55  
USD        $89,533,569.90  
MXN         $5,470,484.60  


---

# Ejercicio 2

En el modelo de datos tenemos dos tablas en SQL con la siguiente estructura

* Tabla ``CLIENTES`` consta de 5 columnas: NUMERO_CLIENTE, MONEDA, CAPITAL_MO, NOMBRE_CLIENTE.
* Tabla ``NEGOCIO`` consta de 2 columnas: NUMERO_CLIENTE y NEGOCIO

Ambas tablas comparten la columna NUMERO_CLIENTE, y ésta es una llave primaria.

Lo que deberás hacer es lo siguiente

a. Realiza una consulta para ver la información de dichas tablas

In [110]:
# solucion

#Función de resguardo para ejecutar querys
def Query(query):
    #Contectamos con la base de datos
    con = sql.connect('BBDD_MUESTRA.db')
    cur = con.cursor()
    #Manejo de excepciones
    try:
        cur.execute(query)
        con.commit()
        print("Query successful")
    except sql.Error as er:
        print(f"Error: '{er}'")

    #devolvemos tuples resultantes de la consulta
    rows = cur.fetchall()
    #Imprimimos los tuples/rows
    for row in rows:
        print(row)

    #Cerramos conexión
    con.close()

#Query's
query1 = """
  SELECT * FROM CLIENTES
"""
query2 = """
  SELECT * FROM NEGOCIO
"""

#Ejecutamos querys para visualizar cada tabla individualmente
Query(query1)
Query(query2)

Query successful
(230, 'EUR', 16595.14, 'Clarke, Walter and Moody')
(907, 'USD', 34577.06, 'Wyatt-Wyatt')
(419, 'EUR', 21747.49, 'Brock-Snyder')
(62, 'EUR', 12962.23, 'Patrick PLC')
(240, 'MXN', 996.61, 'Webb-Day')
(399, 'EUR', 1147.24, 'Summers-Oliver')
(592, 'USD', 6932.67, 'Lamb, Garcia and Thompson')
(789, 'USD', 26668.17, 'Barr and Sons')
(18, 'USD', 32086.69, 'Wilson, Reed and Palmer')
(562, 'USD', 5782.87, 'Rodriguez-Campbell')
(791, 'MXN', 31821.0, 'Mejia, Cowan and Gomez')
(606, 'MXN', 42411.17, 'Fuentes, Hughes and Pitts')
(360, 'MXN', 21775.31, 'Johnston Inc')
(91, 'MXN', 58622.89, 'Ferguson-Gilmore')
(979, 'USD', 34103.74, 'Hughes-Levy')
(581, 'EUR', 67821.32, 'Williams-Walton')
(936, 'EUR', 32824.76, 'Andersen, Hooper and Fritz')
(5, 'EUR', 53844.26, 'Henderson, Reed and Garner')
(746, 'MXN', 22146.16, 'Andersen, Hooper and Fritz')
(757, 'USD', 35542.03, 'Ellis LLC')
(928, 'EUR', 55075.8, 'Gray, Yoder and Contreras')
(107, 'USD', 32035.11, 'Flynn Inc')
(502, 'MXN', 52029.2

b. Une ambas tablas para obtener en la tabla CLIENTES el nogocio proveniente de la tabla NEGOCIO.

In [91]:
# Solucion
#Con USING forzamos a que no se muestre la key de la tabla negocio
queryJoin = """
  SELECT * FROM
  CLIENTES a
  INNER JOIN NEGOCIO b USING (NUMERO_CLIENTE);
"""
#Ejecutamos Query
Query(queryJoin)

Query successful
(230, 'EUR', 16595.14, 'Clarke, Walter and Moody', 'EMPRESAS')
(907, 'USD', 34577.06, 'Wyatt-Wyatt', 'GOBIERNO')
(419, 'EUR', 21747.49, 'Brock-Snyder', 'CORPORATIVA')
(62, 'EUR', 12962.23, 'Patrick PLC', 'EMPRESAS')
(240, 'MXN', 996.61, 'Webb-Day', 'CORPORATIVA')
(399, 'EUR', 1147.24, 'Summers-Oliver', 'CORPORATIVA')
(592, 'USD', 6932.67, 'Lamb, Garcia and Thompson', 'EMPRESAS')
(789, 'USD', 26668.17, 'Barr and Sons', 'EMPRESAS')
(18, 'USD', 32086.69, 'Wilson, Reed and Palmer', 'GOBIERNO')
(562, 'USD', 5782.87, 'Rodriguez-Campbell', 'CORPORATIVA')
(791, 'MXN', 31821.0, 'Mejia, Cowan and Gomez', 'CORPORATIVA')
(606, 'MXN', 42411.17, 'Fuentes, Hughes and Pitts', 'EMPRESAS')
(360, 'MXN', 21775.31, 'Johnston Inc', 'EMPRESAS')
(91, 'MXN', 58622.89, 'Ferguson-Gilmore', 'GOBIERNO')
(979, 'USD', 34103.74, 'Hughes-Levy', 'EMPRESAS')
(581, 'EUR', 67821.32, 'Williams-Walton', 'GOBIERNO')
(936, 'EUR', 32824.76, 'Andersen, Hooper and Fritz', 'EMPRESAS')
(5, 'EUR', 53844.26, 'Hender

c. ¿Cuál es el negocio que tiene la mayor concentración de capital?

In [113]:
# Solucion

queryJoin = """
  WITH CLIENTESMX AS (
    SELECT
        NUMERO_CLIENTE,
        MONEDA,
        CAPITAL_MO,
        NOMBRE_CLIENTE,
        CASE
            WHEN MONEDA = 'EUR' THEN CAPITAL_MO * 18.305021
            WHEN MONEDA = 'USD' THEN CAPITAL_MO * 16.9623
            ELSE CAPITAL_MO
        END AS CAPITAL_VALORIZADO
    FROM
        CLIENTES
)

  SELECT ROUND(SUM(CAPITAL_MO),2) as CAPITAL, ROUND(SUM(CAPITAL_VALORIZADO),2) AS CAPITAL_MXN, NEGOCIO FROM
  CLIENTESMX
  INNER JOIN NEGOCIO USING (NUMERO_CLIENTE)
  GROUP BY NEGOCIO
  ORDER BY CAPITAL_MXN DESC;
"""
#Ejecutamos Query
Query(queryJoin)

#R: El negocio con mayor concentración de capital (tanto en moneda de origen como con la conversión por tipo de cambio es 'CORPORATIVA'

Query successful
(5912518.61, 72205301.15, 'CORPORATIVA')
(5384549.61, 66364844.3, 'GOBIERNO')
(5193716.68, 61539767.6, 'EMPRESAS')
